In [183]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc as misc
%matplotlib inline

#Reading in the data
data = np.load('MC_500_5_4.npz.npy')
trueCls = np.load('MC_500_5_4_reference_classes.npy')

In [194]:
# Reconfigure data to resemble that used in Liu and Wang
r = np.argmax(data,axis=2)

## EM with Single Confusion Matrix

In [195]:
# Number of data points
num_data     = data.shape[0]
# Number of experts performing classification
num_experts  = data.shape[1]
# Number of classes
num_classes  = data.shape[2]

# Initialize rho, the proportion of each class, to be even.  
rho = np.repeat(1./num_classes, num_classes)

# Initialize the shared confusion matrix with a bias lambda
lambdaa = 10.
theta = np.ones((num_classes,num_classes))*(1./(lambdaa+num_classes))
np.fill_diagonal(theta, np.repeat((lambdaa/(lambdaa+num_classes)),num_classes))

In [196]:
def E_step(rho=rho, theta=theta, r=r):
    zeds = np.zeros((num_data, num_classes))
    
    for i in range(num_data):
        for k in range(num_classes):
            log_numerator = np.log(rho[k]) + np.sum([[np.sum(r[i,j]==t)*np.log(theta[k,t]) \
                                                      for j in range(num_experts)] for t in range(num_classes)])
            log_denominator = misc.logsumexp([np.log(rho[k_p]) + np.sum([[np.sum(r[i,j]==t)*np.log(theta[k_p,t]) \
                                                for j in range(num_experts)] for t in range(num_classes)]) \
                                              for k_p in range(num_classes)])
            zeds[i,k] = np.exp(log_numerator - log_denominator)
    return zeds

In [197]:
def M_step(z,r=r):
    thet = np.zeros((num_classes,num_classes))
    for k in range(num_classes):
        for t in range(num_classes):
            numerator=0
            denominator=0
            for i in range(num_data):
                for j in range(num_classes):
                    numerator += np.sum(z[i,k][r[i,j]==t])
                    denominator += np.sum([np.sum(z[i,k][r[i,j]==t_p]) for t_p in range(num_classes)])
  
            thet[k,t] = numerator/denominator
    
    rh = np.zeros(num_classes)
    for k in range(num_classes):
        rh[k] = np.sum(z[:,k])/np.sum(z)
            
    return thet, rh

In [198]:
# Now run it:

# Nonsense initial values to test convergence
z_old = 500
z = 100
ctr = 0

while np.sum(z_old - z) > .00001:
    z_old = z
    z = E_step()
    theta, rho = M_step(z)
    ctr+=1
    
print "Converged within {} iterations".format(ctr)

Converged within 2 iterations


In [199]:
# Shared Confusion Matrix
theta

array([[ 0.76150469,  0.08909535,  0.06398059,  0.08541936],
       [ 0.08197992,  0.75877687,  0.06564687,  0.09359634],
       [ 0.10589271,  0.07680065,  0.7269108 ,  0.09039583],
       [ 0.0931993 ,  0.08032983,  0.08482246,  0.7416484 ]])

In [200]:
# Class distribution
rho

array([ 0.28693228,  0.24531324,  0.22015938,  0.24759511])

In [201]:
print "Accuracy is {}%".format(100.*np.sum(np.argmax(z,axis=1)==trueCls)/num_data)

Accuracy is 95.4%


## EM, multiple confusion matrices

In [338]:
# Re-initialize variables

# Initialize rho, the proportion of each class, to be even.  
rho = np.repeat(1./num_classes, num_classes)

# Initialize the confusion matrices with a bias lambda
lambdaa = 10.
theta = np.ones((num_classes,num_classes))*(1./(lambdaa+num_classes))
np.fill_diagonal(theta, np.repeat((lambdaa/(lambdaa+num_classes)),num_classes))

# All the confusion matrices start out the same
thetas = np.stack([theta]*num_experts)

In [339]:
def E_step2(rho=rho, r=r, thetas=thetas):
    zeds = np.zeros((num_data, num_classes))
    
    for i in range(num_data):
        for k in range(num_classes):
            log_numerator = np.log(rho[k]) + np.sum([thetas[j,k,r[i,j]] for j in range(num_experts)])
            log_denominator = misc.logsumexp([np.log(rho[k_p]) + np.sum([thetas[j,k_p,r[i,j]]\
                                                                       for j in range(num_experts)]) \
                                                                          for k_p in range(num_classes)])
            zeds[i,k] = np.exp(log_numerator - log_denominator)
            
    return zeds

In [340]:
def M_step2(z, r=r):
    thet = np.zeros((num_experts,num_classes,num_classes))
    for k in range(num_classes):
        for t in range(num_classes):
            for j in range(num_experts):
                numerator=0
                denominator=0
                for i in range(num_data):
                    numerator += np.sum(z[i,k][r[i,j]==t])
                    denominator += np.sum([np.sum(z[i,k][r[i,j]==t_p]) for t_p in range(num_classes)])
                thet[j,k,t] = numerator/denominator
    
    rh = np.zeros(num_classes)
    for k in range(num_classes):
        rh[k] = np.sum(z[:,k])/np.sum(z)
            
    return thet, rh
                    

In [341]:
# Now run it:

# Nonsense initial values to test convergence
z_old = 500
z = 100
ctr = 0

while np.sum(z_old - z) > 0.00001:
    z_old = z
    z = E_step2()
    thetas, rho = M_step2(z)
    ctr+=1
    
print "Converged within {} iterations".format(ctr)

Converged within 2 iterations


In [342]:
# Confusion matrices
print thetas

[[[ 0.69586724  0.10046301  0.08668439  0.11698537]
  [ 0.11560531  0.67960661  0.09439964  0.11038844]
  [ 0.13372311  0.11423286  0.62880342  0.12324062]
  [ 0.12526173  0.11142521  0.08899527  0.67431779]]

 [[ 0.58219459  0.13901433  0.14893871  0.12985237]
  [ 0.14852632  0.55547115  0.12118436  0.17481817]
  [ 0.14903173  0.17638159  0.51038099  0.16420569]
  [ 0.14801443  0.15182998  0.14116231  0.55899328]]

 [[ 0.52169192  0.1916419   0.13169312  0.15497306]
  [ 0.18770003  0.52783523  0.13177481  0.15268993]
  [ 0.22080018  0.15011549  0.48123126  0.14785308]
  [ 0.20682207  0.14505233  0.1680955   0.4800301 ]]

 [[ 0.70109614  0.09319384  0.08865878  0.11705124]
  [ 0.13478066  0.64276072  0.10001071  0.12244791]
  [ 0.15928062  0.09469493  0.59534841  0.15067604]
  [ 0.14132006  0.11771149  0.0955745   0.64539394]]

 [[ 0.54178511  0.12738843  0.16086648  0.16995998]
  [ 0.1519544   0.54306519  0.14865858  0.15632182]
  [ 0.16132996  0.14554942  0.54273358  0.15038703]
  [ 

In [343]:
# Class distribution
rho

array([ 0.27552639,  0.24808812,  0.22656323,  0.24982226])

In [344]:
print "Accuracy is {}%".format(100.*np.sum(np.argmax(z,axis=1)==trueCls)/num_data)

Accuracy is 94.8%
